In [1]:
import os   
from dotenv import load_dotenv
load_dotenv()

# agentId = os.getenv('BEDROCK_AGENT_ID')
# agentAliasId = os.getenv('BEDROCK_AGENT_ALIAS_ID')
region_name = 'us-east-1'
knowledgeBaseId = os.getenv('KNOWLEDGEBASEID')
roleArn = os.getenv("BEDROCKAGENTROLE")


In [2]:
import boto3
import uuid,json
from helper import *

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-east-1')

In [4]:
create_agent_response = bedrock_agent.create_agent(
    agentName = "Customer-support-bot-for-e-commerce-platform",
    foundationModel="anthropic.claude-3-haiku-20240307-v1:0",
    instruction = """Guidelines for Handling Customer Messages
Understand the Customer's Message
Carefully read the message to grasp the issue or question being raised.

Decide the Next Step Based on Urgency and Complexity
Choose the right course of action depending on how complex or serious the issue is.

Use the Right Tools or Escalate
If needed, use tools to handle the request—or pass it on to a human agent.

Summarize Your Actions
Provide a brief and clear explanation of what you’ve understood and what actions you’ve taken for internal tracking.

Issue Handling Based on Type
a) For Complex Issues or When Human Help Is Needed
Escalate the issue to a customer support agent immediately.

Include a clear summary of the customer's query and relevant background.

Let the customer know their issue is being passed to a specialist.

b) For Simple Questions or Basic Product Use
Refer to the official knowledge base for accurate information.

Respond clearly and briefly, based on what’s documented.

If the knowledge base doesn’t have the answer, escalate to a human agent.

Important Rules for Every Interaction
Only share information you are completely sure is correct and verified.

Never guess or assume company policies or procedures.

When in doubt—about the question, answer, or next step—escalate to a human.

Use only the information the customer has given. Don’t make assumptions.

If a question involves unknown policies or decisions, route it to a human agent.

Your main role is to direct the query to the right place—not to solve it unless told otherwise.""",
    agentResourceRoleArn = roleArn,
)

In [5]:
agentId = create_agent_response['agent']['agentId']


In [6]:
wait_for_agent_status(
    agentId=agentId,
    targetStatus="NOT_PREPARED"
)
bedrock_agent.prepare_agent(
    agentId=agentId
)
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='Aliasformyagent',
)
agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.
Waiting for agent status of 'PREPARED'...
Agent status: PREPARED
Agent reached 'PREPARED' status.
Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [7]:
describe_agent_response = bedrock_agent.get_agent(
    agentId=agentId)

In [8]:
describe_agent_response

{'ResponseMetadata': {'RequestId': '6295c82b-59fe-442e-9142-754db17e932a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 04 Aug 2025 18:36:24 GMT',
   'content-type': 'application/json',
   'content-length': '16084',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6295c82b-59fe-442e-9142-754db17e932a',
   'x-amz-apigw-id': 'OyySZFemIAMEHaA=',
   'x-amzn-trace-id': 'Root=1-6890fda8-3b109cd834fedbc618b05851'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:038145523842:agent/HO1K1KLV0M',
  'agentCollaboration': 'DISABLED',
  'agentId': 'HO1K1KLV0M',
  'agentName': 'Customer-support-bot-for-e-commerce-platform',
  'agentResourceRoleArn': 'arn:aws:iam::038145523842:role/Agen-access-for-local',
  'agentStatus': 'PREPARED',
  'clientToken': 'd22c42a9-4293-413a-b963-30f1eff8b4c5',
  'createdAt': datetime.datetime(2025, 8, 4, 18, 35, 31, 937013, tzinfo=tzutc()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds

In [9]:
print(json.dumps(describe_agent_response, indent=4, default=str))

{
    "ResponseMetadata": {
        "RequestId": "6295c82b-59fe-442e-9142-754db17e932a",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "date": "Mon, 04 Aug 2025 18:36:24 GMT",
            "content-type": "application/json",
            "content-length": "16084",
            "connection": "keep-alive",
            "x-amzn-requestid": "6295c82b-59fe-442e-9142-754db17e932a",
            "x-amz-apigw-id": "OyySZFemIAMEHaA=",
            "x-amzn-trace-id": "Root=1-6890fda8-3b109cd834fedbc618b05851"
        },
        "RetryAttempts": 0
    },
    "agent": {
        "agentArn": "arn:aws:bedrock:us-east-1:038145523842:agent/HO1K1KLV0M",
        "agentCollaboration": "DISABLED",
        "agentId": "HO1K1KLV0M",
        "agentName": "Customer-support-bot-for-e-commerce-platform",
        "agentResourceRoleArn": "arn:aws:iam::038145523842:role/Agen-access-for-local",
        "agentStatus": "PREPARED",
        "clientToken": "d22c42a9-4293-413a-b963-30f1eff8b4c5",
        "c

In [10]:
get_knowledge_base_response = bedrock_agent.get_knowledge_base(
    knowledgeBaseId=knowledgeBaseId
)

In [11]:
get_knowledge_base_response

{'ResponseMetadata': {'RequestId': '7eff2eb9-5b92-4635-b30e-a58ce5220e09',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 04 Aug 2025 18:37:22 GMT',
   'content-type': 'application/json',
   'content-length': '1050',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7eff2eb9-5b92-4635-b30e-a58ce5220e09',
   'x-amz-apigw-id': 'OyybYGhFIAMEhJQ=',
   'x-amzn-trace-id': 'Root=1-6890fde2-31792f03269ecd5e74d1f4fc'},
  'RetryAttempts': 0},
 'knowledgeBase': {'createdAt': datetime.datetime(2025, 8, 4, 17, 50, 14, 677756, tzinfo=tzutc()),
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:038145523842:knowledge-base/QASQPULXBM',
  'knowledgeBaseConfiguration': {'type': 'VECTOR',
   'vectorKnowledgeBaseConfiguration': {'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/cohere.embed-english-v3',
    'embeddingModelConfiguration': {'bedrockEmbeddingModelConfiguration': {'embeddingDataType': 'FLOAT32'}}}},
  'knowledgeBaseId': 'QASQPULXBM',
  'name': 'knowledge-base-quic

In [12]:
associate_agent_knowledge_base_response = bedrock_agent.associate_agent_knowledge_base(
    agentId=agentId,
    knowledgeBaseId=knowledgeBaseId,
    agentVersion= "DRAFT",
    description="demo-kb-for-e-commerce-bot")

In [13]:
associate_agent_knowledge_base_response

{'ResponseMetadata': {'RequestId': '38710e4e-2065-47b0-b6c9-389bdf616f41',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 04 Aug 2025 18:38:09 GMT',
   'content-type': 'application/json',
   'content-length': '219',
   'connection': 'keep-alive',
   'x-amzn-requestid': '38710e4e-2065-47b0-b6c9-389bdf616f41',
   'x-amz-apigw-id': 'OyyitFUzIAMEbQA=',
   'x-amzn-trace-id': 'Root=1-6890fe10-409f46d246c2813909aab7cc'},
  'RetryAttempts': 0},
 'agentKnowledgeBase': {'createdAt': datetime.datetime(2025, 8, 4, 18, 38, 9, 47626, tzinfo=tzutc()),
  'description': 'demo-kb-for-e-commerce-bot',
  'knowledgeBaseId': 'QASQPULXBM',
  'knowledgeBaseState': 'ENABLED',
  'updatedAt': datetime.datetime(2025, 8, 4, 18, 38, 9, 47626, tzinfo=tzutc())}}

In [14]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='Aliasformyagent',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARED
Agent reached 'PREPARED' status.
Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [15]:
sessionId = str(uuid.uuid4())
message="hey there am tom with email tom@tom.com, i purchase a iphone from your store , and it is damged , can you help me with it , i want a refund ?"

In [16]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,  
    sessionId=sessionId,
    enableTrace=True
)

User: hey there am tom with email tom@tom.com, i purchase a iphone from your
store , and it is damged , can you help me with it , i want a refund ?

Agent: 
Invocation Input:
  Type: KNOWLEDGE_BASE

Observation:
  Type: KNOWLEDGE_BASE
  Knowledge Base Lookup:

Observation:
  Type: FINISH

Final response:
  Sorry, I am unable to assist you with this request.


Session ID: 0f0175e6-b1c5-4ed6-be06-eb1d991c85ae


In [22]:
sessionId = str(uuid.uuid4())
message="hey whats your return policy"

invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,  
    sessionId=sessionId,
    enableTrace=True
)

User: hey whats your return policy

Agent: 
Invocation Input:
  Type: KNOWLEDGE_BASE

Observation:
  Type: KNOWLEDGE_BASE
  Knowledge Base Lookup:
    - ChaoticCart: The Most Questionable E-Commerce Supp...

Observation:
  Type: FINISH

Final response:
  According to the search results, ChaoticCart's return policy is that
  they only accept returns in the form of riddles, mailed via carrier
  pigeons. They also state that bonus points are awarded if the riddle
  rhymes.


Session ID: 1d6dbe66-e49b-48b7-8d77-20a70782f932
